In [0]:
from pyspark.sql.functions import current_timestamp

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/sales/sales/job_if_else/Data_Lake/FAIL/")

record_count = df.count()
print("Record count:", record_count)
if record_count > 0:
    df = df.withColumn("ingestion_time", current_timestamp())

    df.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("sales.sales.sales_raw")

    dbutils.jobs.taskValues.set(key="record_status", value="HAS_DATA")
else:
    dbutils.jobs.taskValues.set(key="record_status", value="NO_DATA")

#{{tasks.01_ingest_data.values.record_status}}== "NO_DATA"
#{{tasks.01_ingest_data.values.record_status}} == "HAS_DATA"
